Antes de comenzar con el codigo importamos las librerias que vamos a usar

In [1]:
# Importar las librerías necesarias
from scapy.all import *
from scapy.layers.inet import IP, ICMP
import time
import statistics

# **Ejercicio 2:**

Pensamos una explicacion bien simple de que es lo que hace el *ping*

**Lo que hace el *ping* para un solo paquete es lo siguiente:**
1. Crear Paquete
2. Enviar Paquete
3. Esperar Respuesta
4. Mostrar Respuesta

Para que lo haga varias veces, se puede hacer un bucle que repita los pasos 1 a 4 la cantidad de veces que se quiera

# **Desarrollo del código**:

Para desarrollar el código, lo primero que hicimos fue tomar la recomendación del punto c. la de crear la función de ping para un solo paquete.

In [2]:
def ping(host):

    tamano = int(input("Ingrese los bytes del payload (default 28 de encabezado del paquete): ")) # Linea utilizada para modificar el payload del paquete y hacerlo personalizado. Tener en cuenta que esto va a afectar el payload, tenemos  28B de base, viene de 20B de IP y 8B de ICMP. Por lo que pbtendremos del tamano del paquete sera 28B + el payload que elija el usuario.
    payload = b"A" * tamano # Crear un payload de tamaño especificado

    print(f"Ping {host}")
    # Inicializar contadores
    paquetes_enviados = 1
    paquetes_recibidos = 0
    paquetes_perdidos = 0

    # 1. Crear Paquete
    # paquete = IP(dst=host)/ICMP() # Crea un paquete IP con destino al host especificado y un paquete ICMP (que es el protocolo utilizado por ping) con un paquete default de 28B, esto se obtiene ya que tenemos 20B de IP y 8B de ICMP
    paquete = IP(dst=host)/ICMP()/payload # Crea un paquete IP con destino al host especificado y un paquete ICMP (que es el protocolo utilizado por ping) con un payload perosnalizado por el usuario
    
    # 2. Enviar Paquete
    inicio = time.time()
    respuesta = sr1(paquete, timeout=1, verbose=False)  #  sr1 me devuelve el paquete de respuesta o None si no hay respuesta, verbose=False, se usa para ocultar mensajes internos de Scapy.
    
    # 3. Esperar Respuesta
    if (respuesta is None):
        print(f"Tiempo de espera agotado (timeout) para el host {host}")
        paquetes_perdidos = 1
        porcentaje_perdidos = 100
        
    else:
        # 4. Mostrar Respuesta
        fin = time.time()
        rtt = (fin - inicio) * 1000 # Convertir a milisegundos
        paquetes_recibidos = 1
        paquetes_perdidos = 0
        porcentaje_perdidos = 0
        tamano_paquete = len(paquete)
        print(f"Respuesta de {host}: Tiempo = {rtt} ms, Tamaño = {tamano_paquete} bytes, TTL = {respuesta.ttl} Hops")

    if (paquetes_recibidos == 1):
        porcentaje_perdidos = 0
    else:
        porcentaje_perdidos = 100

    # Imprimo el resumen    
    print("\n--- Estadísticas de ping ---")
    print(f"Cantidad de paquetes enviados: {paquetes_enviados}")
    print(f"Cantidad de paquetes recibidos: {paquetes_recibidos}")
    print(f"Cantidad de paquetes perdidos: {paquetes_perdidos}")
    print(f"Porcentaje de paquetes perdidos: {porcentaje_perdidos}%")
    
    if (porcentaje_perdidos == 0):
        print(f"RTT promedio: {rtt} ms")
    else:
        print("No se recibieron respuestas, no hay estadísticas de RTT.")

# Ejemplo de uso:
ping("google.com") # Me va a hacer ping a google.com, que es una IP valida por lo que deberia responder todo bien. Puedo recibir la respuesta de que llego el paquete o un timeout

Ping google.com
Respuesta de google.com: Tiempo = 81.77304267883301 ms, Tamaño = 32 bytes, TTL = 249 Hops

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 1
Cantidad de paquetes perdidos: 0
Porcentaje de paquetes perdidos: 0%
RTT promedio: 81.77304267883301 ms


Luego, lo que hicimos fue agregar un *bucle, un **for*** para así lograr enviar varios paquetes y modificar las variables necesarias para que el código quede consistente y funcione correctamente

In [3]:
def ping(host):

    cantidad = int(input("Ingrese la cantidad de paquetes a enviar: ")) 
    tamano = int(input("Ingrese los bytes del payload (default 28 de encabezado del paquete): ")) # Linea utilizada para modificar el payload del paquete y hacerlo personalizado. Tener en cuenta que esto va a afectar el payload, tenemos  28B de base, viene de 20B de IP y 8B de ICMP. Por lo que pbtendremos del tamano del paquete sera 28B + el payload que elija el usuario.
    payload = b"A" * tamano # Crear un payload de tamaño especificado
    
    print(f"\n ")
    print(f"Ping {host}")
    if (cantidad <= 0):
        print("La cantidad de paquetes debe ser un número positivo y mayor a cero.")
        return
    elif (cantidad == 1):
        print(f"Enviando {cantidad} paquete a {host}...\n")
    else:   
        print(f"Enviando {cantidad} paquetes a {host}...\n")

    # Inicializo una variable para ir contando los paquetes enviados
    paquetes_enviados = 0

    # Inicializo una variable para ir contando los paquetes recibidos
    paquetes_recibidos = 0

    # Creo un vector para los rtt (round trip time)
    rtts = []
    
    for i in range(cantidad):
        
        # 1. Crear Paquete
        paquetes_enviados += 1
        # paquete = IP(dst=host)/ICMP() # Crea un paquete IP con destino al host especificado y un paquete ICMP (que es el protocolo utilizado por ping) con un paquete default de 28B, esto se obtiene ya que tenemos 20B de IP y 8B de ICMP
        paquete = IP(dst=host)/ICMP()/payload # Crea un paquete IP con destino al host especificado y un paquete ICMP (que es el protocolo utilizado por ping) con un payload perosnalizado por el usuario
        
        # 2. Enviar Paquete
        inicio = time.time()
        respuesta = sr1(paquete, timeout=1, verbose=False) # sr1 me devuelve el paquete de respuesta o None si no hay respuesta, verbose=False, se usa para ocultar mensajes internos de Scapy.
    
        # 3. Esperar Respuesta

        # Si no hay respuesta, se maneja el timeout
        if (respuesta is None):
            print(f"Tiempo de espera agotado (timeout) para el host {host}")

        # Si hay respuesta, se muestra la información
        else:
            # 4. Mostrar Respuesta
            fin = time.time()
            rtt = (fin - inicio) * 1000 # Convertir a milisegundos
            rtts.append(rtt)
            paquetes_recibidos += 1
            tamano_paquete = len(paquete)
            print(f"Respuesta de {host}: Tiempo = {rtt} ms, Tamaño = {tamano_paquete} bytes, TTL = {respuesta.ttl} Hops")
            
    # Cantidad de paquetes perdidos
    paquetes_perdidos = paquetes_enviados - paquetes_recibidos

    # Porcentaje de paquetes perdidios
    if (paquetes_enviados > 0):
        porcentaje_perdidos = (paquetes_perdidos / paquetes_enviados) * 100
    else:
        porcentaje_perdidos = 0

    # Imprimo el resumen
    print("\n--- Estadísticas de ping ---")
    print(f"Cantidad de paquetes enviados: {paquetes_enviados}")
    print(f"Cantidad de paquetes recibidos: {paquetes_recibidos}")
    print(f"Cantidad de paquetes perdidos: {paquetes_perdidos}")
    print(f"Porcentaje de paquetes perdidos: {porcentaje_perdidos}%")

    if (len(rtts) > 0):
        print(f"RTT promedio: {statistics.mean(rtts)} ms")
        print(f"RTT máximo: {max(rtts)} ms")
        print(f"RTT mínimo: {min(rtts)} ms")
        if len(rtts) > 1:
            print(f"Desvío estándar del RTT: {statistics.stdev(rtts)} ms")
        else:
            print("Desvío estándar del RTT: N/A (solo un paquete recibido)")
    else:
        print("No se recibieron respuestas, no hay estadísticas de RTT.")

ping("google.com")  # Me va a hacer ping a google.com, que es una IP valida por lo que deberia responder todo bien. Puedo recibir la respuesta de que llego el paquete o un timeout.
ping("192.0.2.1") # Me va a hacer ping a 192.0.2.1, que es una IP invalida por lo que deberia responder timeout, ya que en esta parte del codigo no esta implementado el manejo de errores de tipo 3.


 
Ping google.com
Enviando 2 paquetes a google.com...

Respuesta de google.com: Tiempo = 38.4669303894043 ms, Tamaño = 32 bytes, TTL = 249 Hops
Respuesta de google.com: Tiempo = 28.93233299255371 ms, Tamaño = 32 bytes, TTL = 249 Hops

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 2
Cantidad de paquetes recibidos: 2
Cantidad de paquetes perdidos: 0
Porcentaje de paquetes perdidos: 0.0%
RTT promedio: 33.699631690979004 ms
RTT máximo: 38.4669303894043 ms
RTT mínimo: 28.93233299255371 ms
Desvío estándar del RTT: 6.741978475196653 ms

 
Ping 192.0.2.1
Enviando 2 paquetes a 192.0.2.1...

Tiempo de espera agotado (timeout) para el host 192.0.2.1
Tiempo de espera agotado (timeout) para el host 192.0.2.1

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 2
Cantidad de paquetes recibidos: 0
Cantidad de paquetes perdidos: 2
Porcentaje de paquetes perdidos: 100.0%
No se recibieron respuestas, no hay estadísticas de RTT.


# **Ejercicio 3:**

Para ver e interpretar cada uno de los errores en *ICMP* de tipo 3, utilizamos el comando *.getlayer*, que se encarga de obtener la etiqueta del tipo del error (En este caso 3). Y ademas utilizaremos *haslayer* para ver que efectivamente el paquete tenga la etiqueta de error *ICMP 3*

In [4]:
def ping(host):

    cantidad = int(input("Ingrese la cantidad de paquetes a enviar: ")) 
    tamano = int(input("Ingrese los bytes del payload (default 28 de encabezado del paquete): ")) # Linea utilizada para modificar el payload del paquete y hacerlo personalizado. Tener en cuenta que esto va a afectar el payload, tenemos  28B de base, viene de 20B de IP y 8B de ICMP. Por lo que pbtendremos del tamano del paquete sera 28B + el payload que elija el usuario.
    payload = b"A" * tamano # Crear un payload de tamaño especificado
    
    print(f"\n ")
    print(f"Ping {host}")
    # Si el usuario puso para enviar cero paquetes o un numero positivo, se le informa que no es correcto y se le vuelve a pedir que ingrese la cantidad de paquetes a enviar.
    if (cantidad <= 0):
        print("La cantidad de paquetes debe ser un número positivo y mayor a cero.")
        return
    elif (cantidad == 1):
        print(f"Enviando {cantidad} paquete a {host}...\n")
    else:   
        print(f"Enviando {cantidad} paquetes a {host}...\n")

    # Inicializo una variable para ir contando los paquetes enviados
    paquetes_enviados = 0

    # Inicializo una variable para ir contando los paquetes recibidos
    paquetes_recibidos = 0

    # Creo un vector para los rtt (round trip time)
    rtts = []
    
    for i in range(cantidad):
        
        # 1. Crear Paquete
        paquetes_enviados += 1
        # paquete = IP(dst=host)/ICMP() # Crea un paquete IP con destino al host especificado y un paquete ICMP (que es el protocolo utilizado por ping) con un paquete default de 28B, esto se obtiene ya que tenemos 20B de IP y 8B de ICMP
        paquete = IP(dst=host)/ICMP()/payload # Crea un paquete IP con destino al host especificado y un paquete ICMP (que es el protocolo utilizado por ping) con un payload perosnalizado por el usuario
        
        # 2. Enviar Paquete
        inicio = time.time()
        respuesta = sr1(paquete, timeout=1, verbose=False) # sr1 me devuelve el paquete de respuesta o None si no hay respuesta, verbose=False, se usa para ocultar mensajes internos de Scapy.
    
        # 3. Esperar Respuesta

        # Si no hay respuesta, se maneja el timeout
        if (respuesta is None):
            print(f"Tiempo de espera agotado (timeout) para el host {host}")

        # Si hay respuesta, se muestra la información
        else:
            
            # Chequeo si el tipo de respuesta es un ICMP de tipo 3.
            if (respuesta.haslayer(ICMP) and respuesta.getlayer(ICMP).type == 3):
                codigo = respuesta.getlayer(ICMP).code
                errores = {
                    0: "Red inalcanzable",
                    1: "Host inalcanzable",
                    2: "Protocolo inalcanzable",
                    3: "Puerto inalcanzable",
                    13: "Comunicación prohibida por administrador"
                }
                mensaje = errores.get(codigo, f"Código desconocido ({codigo})")
                print(f"Error ICMP tipo 3 recibido de {host}: {mensaje}")
                return

            else:
                # 4. Mostrar Respuesta
                fin = time.time()
                rtt = (fin - inicio) * 1000 # Convertir a milisegundos
                rtts.append(rtt)
                paquetes_recibidos += 1
                tamano_paquete = len(paquete)
                print(f"Respuesta de {host}: Tiempo = {rtt} ms, Tamaño = {tamano_paquete} bytes, TTL = {respuesta.ttl} Hops")

    # Cantidad de paquetes perdidos
    paquetes_perdidos = paquetes_enviados - paquetes_recibidos

    # Porcentaje de paquetes perdidios
    if (paquetes_enviados > 0):
        porcentaje_perdidos = (paquetes_perdidos / paquetes_enviados) * 100
    else:
        porcentaje_perdidos = 0

    # Imprimo el resumen
    print("\n--- Estadísticas de ping ---")
    print(f"Cantidad de paquetes enviados: {paquetes_enviados}")
    print(f"Cantidad de paquetes recibidos: {paquetes_recibidos}")
    print(f"Cantidad de paquetes perdidos: {paquetes_perdidos}")
    print(f"Porcentaje de paquetes perdidos: {porcentaje_perdidos}%")

    if (len(rtts) > 0):
        print(f"RTT promedio: {statistics.mean(rtts)} ms")
        print(f"RTT máximo: {max(rtts)} ms")
        print(f"RTT mínimo: {min(rtts)} ms")
        if len(rtts) > 1:
            print(f"Desvío estándar del RTT: {statistics.stdev(rtts)} ms")
        else:
            print("Desvío estándar del RTT: N/A (solo un paquete recibido)")
    else:
        print("No se recibieron respuestas, no hay estadísticas de RTT.")

ping("google.com") # Me va a hacer ping a google.com, que es una IP valida por lo que deberia responder todo bien. Puedo recibir la respuesta de que llego el paquete o un timeout.
ping("192.0.2.1") # Me va a hacer ping a 192.0.2.1, que es una IP invalida por lo que deberia responder un error de tipo 3. Puedo recibir la respuesta de un timeout, dado que el Rputer filtra estos mensajes, que el Host esta demasiado lejos o que nunca soy bloqueado por un firewall. O puedo recibir un mensaje de error de tipo 3, que indica que el host es inalcanzable.


 
Ping google.com
Enviando 2 paquetes a google.com...

Respuesta de google.com: Tiempo = 55.35888671875 ms, Tamaño = 32 bytes, TTL = 249 Hops
Respuesta de google.com: Tiempo = 78.67026329040527 ms, Tamaño = 32 bytes, TTL = 249 Hops

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 2
Cantidad de paquetes recibidos: 2
Cantidad de paquetes perdidos: 0
Porcentaje de paquetes perdidos: 0.0%
RTT promedio: 67.01457500457764 ms
RTT máximo: 78.67026329040527 ms
RTT mínimo: 55.35888671875 ms
Desvío estándar del RTT: 16.483632452610657 ms

 
Ping 192.0.2.1
Enviando 2 paquetes a 192.0.2.1...

Tiempo de espera agotado (timeout) para el host 192.0.2.1
Tiempo de espera agotado (timeout) para el host 192.0.2.1

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 2
Cantidad de paquetes recibidos: 0
Cantidad de paquetes perdidos: 2
Porcentaje de paquetes perdidos: 100.0%
No se recibieron respuestas, no hay estadísticas de RTT.


# **Testeo de la funcion *ping*:**

## **Pings que me devuelven respuestas exitosas:**

In [5]:
ping("8.8.8.8")         # DNS de Google
ping("1.1.1.1")         # DNS de Cloudflare
ping("google.com")      # Dominio conocido con buena disponibilidad


 
Ping 8.8.8.8
Enviando 1 paquete a 8.8.8.8...

Respuesta de 8.8.8.8: Tiempo = 38.6660099029541 ms, Tamaño = 32 bytes, TTL = 249 Hops

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 1
Cantidad de paquetes perdidos: 0
Porcentaje de paquetes perdidos: 0.0%
RTT promedio: 38.6660099029541 ms
RTT máximo: 38.6660099029541 ms
RTT mínimo: 38.6660099029541 ms
Desvío estándar del RTT: N/A (solo un paquete recibido)

 
Ping 1.1.1.1
Enviando 1 paquete a 1.1.1.1...

Respuesta de 1.1.1.1: Tiempo = 90.7280445098877 ms, Tamaño = 28 bytes, TTL = 248 Hops

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 1
Cantidad de paquetes perdidos: 0
Porcentaje de paquetes perdidos: 0.0%
RTT promedio: 90.7280445098877 ms
RTT máximo: 90.7280445098877 ms
RTT mínimo: 90.7280445098877 ms
Desvío estándar del RTT: N/A (solo un paquete recibido)

 
Ping google.com
Enviando 1 paquete a google.com...

Respuesta de google.com: Tiempo

## **Pings que me devuelven un *timeout*:**

In [6]:
ping("192.0.2.1")       # IP reservada para documentación, no responde
ping("203.0.113.1")     # Otra IP reservada para ejemplos, suele no responder
ping("10.255.255.1")    # IP privada de broadcast, normalmente sin respuesta


 
Ping 192.0.2.1
Enviando 1 paquete a 192.0.2.1...

Tiempo de espera agotado (timeout) para el host 192.0.2.1

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 0
Cantidad de paquetes perdidos: 1
Porcentaje de paquetes perdidos: 100.0%
No se recibieron respuestas, no hay estadísticas de RTT.

 
Ping 203.0.113.1
Enviando 1 paquete a 203.0.113.1...

Tiempo de espera agotado (timeout) para el host 203.0.113.1

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 0
Cantidad de paquetes perdidos: 1
Porcentaje de paquetes perdidos: 100.0%
No se recibieron respuestas, no hay estadísticas de RTT.

 
Ping 10.255.255.1
Enviando 1 paquete a 10.255.255.1...

Tiempo de espera agotado (timeout) para el host 10.255.255.1

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 0
Cantidad de paquetes perdidos: 1
Porcentaje de paquetes perdidos: 100.0%
No se recibieron respuestas,

## **Pings que me devuelven un *Error de tipo 3* o un *timeout* (Me puede devolver *timeout* en lugar de un *Error de tipo 3* por 2 razones):**
1. Mi Router filtra este tipo de respuestas 
2. El Host no existe y no responde

In [7]:
ping("192.168.123.123") # IP privada no existente en la mayoría de las redes
ping("198.18.0.1")      # IP usada para pruebas de benchmark, sin host real
ping("172.31.255.255")  # Broadcast en rango privado, suele generar error ICMP


 
Ping 192.168.123.123
Enviando 1 paquete a 192.168.123.123...

Tiempo de espera agotado (timeout) para el host 192.168.123.123

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 0
Cantidad de paquetes perdidos: 1
Porcentaje de paquetes perdidos: 100.0%
No se recibieron respuestas, no hay estadísticas de RTT.

 
Ping 198.18.0.1
Enviando 1 paquete a 198.18.0.1...

Tiempo de espera agotado (timeout) para el host 198.18.0.1

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 0
Cantidad de paquetes perdidos: 1
Porcentaje de paquetes perdidos: 100.0%
No se recibieron respuestas, no hay estadísticas de RTT.

 
Ping 172.31.255.255
Enviando 1 paquete a 172.31.255.255...

Tiempo de espera agotado (timeout) para el host 172.31.255.255

--- Estadísticas de ping ---
Cantidad de paquetes enviados: 1
Cantidad de paquetes recibidos: 0
Cantidad de paquetes perdidos: 1
Porcentaje de paquetes perdidos: 100.0%
No se r